# Cancer drug synergy prediction project

The aim of this project is to build a regression model to predict the synergy between two drugs given the drugs and the cell line.

The problem has been proposed before in a DREAM challenge:

**AstraZeneca-Sanger Drug Combination Prediction DREAM Challenge**<br>
https://www.synapse.org/#!Synapse:syn4231880/wiki/235645
        

## Data sources:
    
The main data source to obtain drug combination data is DrugCombDB.

**DrugCombDB**: http://drugcombdb.denglab.org/ <br>
publication: https://doi.org/10.1093/nar/gkz1007 <br>
Downloaded Files: <br>
&nbsp;&nbsp;&nbsp;1- Drug combinations scores: http://drugcombdb.denglab.org/download/drugcombs_scored.csv <br> 
&nbsp;&nbsp;&nbsp;2- Drug chemical info: http://drugcombdb.denglab.org/download/drug_chemical_info.csv<br>
&nbsp;&nbsp;&nbsp;3- Cell line info: http://drugcombdb.denglab.org/download/cell_Line.csv <br>

### Complementary data sources (data enrichment)

**1- Pubchem REST API**: https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest <br>

**2- Cell line dataset from Expasy**: https://web.expasy.org/cellosaurus/ <br>
Release 41: March 2022 <br>
FTP folder: https://ftp.expasy.org/databases/cellosaurus <br>
&nbsp;&nbsp;&nbsp;Dowload TXT: https://ftp.expasy.org/databases/cellosaurus/cellosaurus.txt <br>
&nbsp;&nbsp;&nbsp;Download XML: https://ftp.expasy.org/databases/cellosaurus/cellosaurus.xml <br>
&nbsp;&nbsp;&nbsp;Download XSD schema: https://ftp.expasy.org/databases/cellosaurus/cellosaurus.xsd <br>

<br>

**NOTE**: All the files were downloaded to the folder named "data" next to this Jupyter Notebook


### Required packages

* numby
* pandas
* urllib
* requests
* xml
* rdkit

### Import packages

In [1]:
!pip3 install rdkit-pypi

     |████████████████████████████████| 22.7 MB 9.6 MB/s eta 0:00:01     |██████████████▌                 | 10.3 MB 8.0 MB/s eta 0:00:02


In [1]:
import numpy as np
import pandas as pd
import urllib.parse as up
import requests
import xml.etree.ElementTree as ET

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import rdMolDescriptors

### Define a function to obtain the SMILES string by drug cid or name

In [4]:
def get_pubchem_smiles(row, by="cid"):
    
    if (pd.isnull(row['smilesString']) or row['smilesString'].strip() == "" or row['smilesString'].strip() == "none"):
        
        domain = 'cid'
        
        if(by == "cid"):
            ent = row['cIds'][4:]
            domain = 'cid'
        elif(by == "name"):
            ent = row['drugName']
            domain = 'name'

        ent = up.quote(ent.encode("utf-8"))
        
        response = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/"+domain+"/"+ent+"/property/CanonicalSMILES/json")
        data = response.json()
        
        try:
            return data['PropertyTable']['Properties'][0]['CanonicalSMILES']
        except KeyError:
            return np.NaN
    else:
        return row['smilesString']

### Test Pubchem API

In [3]:
def test(cid="00065628"):
    
    response = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/"+cid+"/property/CanonicalSMILES/json")
    data = response.json()
    
    print(data['PropertyTable']['Properties'][0]['CanonicalSMILES'])

In [4]:
test()

CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O


## Load and prepare the drugs chemcial info dataframe 

In [5]:
drug_info = pd.read_csv('data/drug_chemical_info.csv', sep=',', dtype='string')

In [6]:
drug_info = drug_info.drop('drugNameOfficial', 1)
drug_info = drug_info.drop('molecularWeight', 1)

### Fill missing SMILES strings using Pubchem API

In [7]:
drug_info['smilesString'] = drug_info.apply(get_pubchem_smiles, by="cid", axis=1)

In [18]:
drug_info = drug_info[drug_info['smilesString'].notna()] # 3 drugs were lost here since SMILES couldn't be obtained for them

In [21]:
drug_info.head()

,drugName,smilesString
0,Bendamustine,CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O
1,Lonidamine,C1=CC=C2C(=C1)C(=NN2CC3=C(C=C(C=C3)Cl)Cl)C(=O)O
2,Lenalidomide,C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N
3,Cladribine,C1C(C(OC1N2C=NC3=C2N=C(N=C3N)Cl)CO)O
4,Pentostatin,C1C(C(OC1N2C=NC3=C2NC=NCC3O)CO)O


In [20]:
drug_info.to_csv('output/drug_chemical_info_complete.csv', sep=',', index=False, encoding="utf-8")

### We want to check if the dug combination scores dataframe contains more drugs than the drug chemical info dataframe

In [21]:
drug_ref_set = set(drug_info['drugName'].str.lower().dropna().tolist())

In [22]:
len(drug_ref_set)

3056

In [23]:
drugcomb_info = pd.read_csv('data/drugcombs_scored.csv', sep=',', dtype='string')

In [24]:
drugs1 = set(drugcomb_info['Drug1'].str.lower().dropna().tolist())
drugs2 = set(drugcomb_info['Drug2'].str.lower().dropna().tolist())
drug_comb_set = drugs1 | drugs2 #union

In [25]:
len(drug_comb_set)

5351

### Apparently, there is 2312 more drugs in the drugcomp scores dataframe than the drug chemical info dataframe

In [26]:
missing_drugs = drug_comb_set - drug_ref_set

In [27]:
len(missing_drugs)

2315

### Lets create a dataframe from the missing drugs and obtiain their SMILES from Pubchem API by name

In [28]:
missing_drugs_df = pd.DataFrame(missing_drugs, columns=['drugName'])

In [29]:
missing_drugs_df['smilesString'] = "none"

In [30]:
missing_drugs_df['smilesString'] = missing_drugs_df.apply(get_pubchem_smiles, by="name", axis=1)

In [34]:
missing_drugs_df = missing_drugs_df[missing_drugs_df['smilesString'].notna()]

In [35]:
missing_drugs_df.shape

(2238, 2)

In [36]:
missing_drugs_df.head()

,drugName,smilesString
0,l-(+)-rhamnose monohydrate,CC(C(C(C(C=O)O)O)O)O.O
1,gsk2606414,CN1C=C(C2=C(N=CN=C21)N)C3=CC4=C(C=C3)N(CC4)C(=...
2,mdivi-1,COC1=C(C=C(C(=C1)N2C(=O)C3=CC=CC=C3NC2=S)Cl)Cl
3,sb705498,C1CN(CC1NC(=O)NC2=CC=CC=C2Br)C3=NC=C(C=C3)C(F)...
4,sgc 0946,CC(C)N(CCCNC(=O)NC1=CC=C(C=C1)C(C)(C)C)CC2C(C(...


In [37]:
missing_drugs_df.to_csv('output/missing_drug_chemical_smiles.csv', sep=',', index=False, encoding="utf-8")

### Now, lets combine the original drug info dataframe and the new missing drugs dataframe

In [38]:
drug_info = drug_info[["drugName", "smilesString"]].append(missing_drugs_df, ignore_index=True)

In [39]:
drug_info.shape

(5294, 2)

In [40]:
drug_info.to_csv('output/all_drugs_with_smiles.csv', sep=',', index=False, encoding="utf-8")

In [41]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(drug_info.smilesString)

0              CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O
1         C1=CC=C2C(=C1)C(=NN2CC3=C(C=C(C=C3)Cl)Cl)C(=O)O
2                   C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N
3                    C1C(C(OC1N2C=NC3=C2N=C(N=C3N)Cl)CO)O
4                        C1C(C(OC1N2C=NC3=C2NC=NCC3O)CO)O
5                                  C1=NC2=C(N1)C(=S)N=CN2
6       CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...
7                        CCC1(CCC(=O)NC1=O)C2=CC=C(C=C2)N
8                 CC12CCC(=O)C(=C1CCC3C2CCC4(C3CCC4=O)C)O
9            C1=CC(=CC=C1C#N)C(C2=CC=C(C=C2)C#N)N3C=NC=N3
10      C1=CC(=CC=C1CCC2=CNC3=C2C(=O)NC(=N3)N)C(=O)NC(...
11                  C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F
12      CCC(=C(C1=CC=CC=C1)C2=CC=C(C=C2)OCCN(C)C)C3=CC...
13      CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...
14           COC1=C(C=C(C=C1)C=CC2=CC(=C(C(=C2)OC)OC)OC)O
15      CC(C)(C)C1=C(N=CN1)C=C2C(=O)NC(=CC3=CC=CC=C3)C...
16      CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...
17           C

In [3]:
drug_info = pd.read_csv('output/all_drugs_with_smiles.csv', sep=',', dtype='string')

## Load and prepare the cell lines info dataframe

In [42]:
cellLine_info = pd.read_csv('data/cell_Line.csv', sep=',', dtype='string')

In [43]:
cellLine_info.shape

(104, 3)

In [44]:
cellLine_info['category'] = 'none'
cellLine_info['gender'] = 'none'
cellLine_info['age'] = 'none'
cellLine_info = cellLine_info.drop('tag', 1)

In [45]:
cellLine_info.head()

,cellName,cosmicId,category,gender,age
0,786-0,905947,none,none,none
1,A498,905948,none,none,none
2,A549/ATCC,905949,none,none,none
3,ACHN,905950,none,none,none
4,CCRF-CEM,905952,none,none,none


In [46]:
len(set(cellLine_info.cosmicId.tolist()))

88

### Enrich cell lines data with category, gender and age information

In [47]:
tree = ET.parse('data/cellosaurus.xml')
root = tree.getroot()

cellLinesList = root[1]

In [48]:
for cellLine in cellLinesList:
    
    category = ''
    gender = ''
    age = ''
    
    nameListElm = cellLine.find('name-list')
    nameList = set()
    
    for nameElm in nameListElm:
        nameList.add(nameElm.text.lower())     
    nameList = list(nameList)
    
    refs = cellLine.find('xref-list')
    accessionList = set()
    
    if(refs is not None):
        for ref in refs:
            if(ref.get('database') in ["Cosmic","Cosmic-CLP"]):
                accessionList.add(ref.get('accession').strip())
    accessionList = list(accessionList)
    
    if('category' in cellLine.attrib):
        category = cellLine.attrib['category']
    else:
        category = 'Undefined cell line type'
        
    if('sex' in cellLine.attrib):
        gender = cellLine.attrib['sex']
    else:
        gender = "Sex unspecified"
        
    if('age' in cellLine.attrib):
        age = cellLine.attrib['age']
    else:
        age = "Age unspecified"
    
    cellLine_info.loc[(cellLine_info['cellName'].str.lower().isin(nameList)) | (cellLine_info['cosmicId'].isin(accessionList)), 'category'] = category
    cellLine_info.loc[(cellLine_info['cellName'].str.lower().isin(nameList)) | (cellLine_info['cosmicId'].isin(accessionList)), 'gender'] = gender
    cellLine_info.loc[(cellLine_info['cellName'].str.lower().isin(nameList)) | (cellLine_info['cosmicId'].isin(accessionList)), 'age'] = age


### One cell type was not found in Expasy cell lines but found online:
https://www.atcc.org/products/crl-2946

In [49]:
cellLine_info.loc[cellLine_info['cellName'] == "UWB1289BRCA1", 'category'] = "Cancer cell line"
cellLine_info.loc[cellLine_info['cellName'] == "UWB1289BRCA1", 'gender'] = "Female"
cellLine_info.loc[cellLine_info['cellName'] == "UWB1289BRCA1", 'age'] = "56Y"

### After checking the "age" column, some incosistencies were found and fixed

In [50]:
cellLine_info.age.unique()

array(['58Y', '52Y', '22Y', '3Y11M', '70Y', '69Y', 'Age unspecified',
       '48Y', '36Y', '60Y', '62Y', '47Y', '53Y', '43Y', '19Y', '64Y',
       '51Y', '42Y', '61Y', '24Y', '67Y', '34Y', '75Y', '11Y', '54Y',
       '30Y6M', '13W', '33Y', '49Y', '74Y', '55Y', 'Adult', '46Y',
       'Blastocyst stage', '29Y', '50Y', '56Y', '27Y', '18Y', '65Y',
       '59Y', '63Y', '39Y', '25-26Y'], dtype=object)

In [51]:
cellLine_info.loc[cellLine_info['age'] == "13W", 'age'] = "1Y"
cellLine_info.loc[cellLine_info['age'] == "Blastocyst stage", 'age'] = "Age unspecified"
cellLine_info.loc[cellLine_info['age'] == "Adult", 'age'] = "Age unspecified"
cellLine_info.loc[cellLine_info['age'] == "25-26Y", 'age'] = "25Y"

cellLine_info.loc[cellLine_info['age'] == "3Y11M", 'age'] = "4Y"
cellLine_info.loc[cellLine_info['age'] == "30Y6M", 'age'] = "31Y"

In [52]:
cellLine_info.age.unique()

array(['58Y', '52Y', '22Y', '4Y', '70Y', '69Y', 'Age unspecified', '48Y',
       '36Y', '60Y', '62Y', '47Y', '53Y', '43Y', '19Y', '64Y', '51Y',
       '42Y', '61Y', '24Y', '67Y', '34Y', '75Y', '11Y', '54Y', '31Y',
       '1Y', '33Y', '49Y', '74Y', '55Y', '46Y', '29Y', '50Y', '56Y',
       '27Y', '18Y', '65Y', '59Y', '63Y', '39Y', '25Y'], dtype=object)

In [53]:
cellLine_info.head()

,cellName,cosmicId,category,gender,age
0,786-0,905947,Cancer cell line,Male,58Y
1,A498,905948,Cancer cell line,Male,52Y
2,A549/ATCC,905949,Cancer cell line,Male,58Y
3,ACHN,905950,Cancer cell line,Male,22Y
4,CCRF-CEM,905952,Cancer cell line,Female,4Y


In [54]:
cellLine_info.to_csv('output/cell_lines_enriched.csv', sep=',', index=False, encoding="utf-8")

In [4]:
cellLine_info = pd.read_csv('output/cell_lines_enriched.csv', sep=',', dtype='string')

## Feature Engineering

### Cell lines: apply One Hot Encoding on category and gender columns and clean the age column

In [5]:
cellLine_info.head()

,cellName,cosmicId,category,gender,age
0,786-0,905947,Cancer cell line,Male,58Y
1,A498,905948,Cancer cell line,Male,52Y
2,A549/ATCC,905949,Cancer cell line,Male,58Y
3,ACHN,905950,Cancer cell line,Male,22Y
4,CCRF-CEM,905952,Cancer cell line,Female,4Y


In [6]:
cat_ohe = pd.get_dummies(cellLine_info.category, prefix='cat')

In [7]:
cellLine_info_full = pd.concat([cellLine_info,cat_ohe], axis=1)

In [8]:
gender_ohe = pd.get_dummies(cellLine_info.gender, prefix='gender')

In [9]:
cellLine_info_full = pd.concat([cellLine_info_full,gender_ohe], axis=1)

In [10]:
cellLine_info_full['age'] = cellLine_info_full['age'].str.replace('Y','').replace('Age unspecified','0')
cellLine_info_full['age'] = cellLine_info_full['age'].astype('int')

In [11]:
cellLine_info_full['age'].unique()

array([58, 52, 22,  4, 70, 69,  0, 48, 36, 60, 62, 47, 53, 43, 19, 64, 51,
       42, 61, 24, 67, 34, 75, 11, 54, 31,  1, 33, 49, 74, 55, 46, 29, 50,
       56, 27, 18, 65, 59, 63, 39, 25])

In [12]:
cellLine_info_full = cellLine_info_full.drop('cosmicId', 1)
cellLine_info_full = cellLine_info_full.drop('category', 1)
cellLine_info_full = cellLine_info_full.drop('gender', 1)

In [13]:
cellLine_info_full.head()

,cellName,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line,gender_Female,gender_Male,gender_Sex unspecified
0,786-0,58,1,0,0,0,0,1,0
1,A498,52,1,0,0,0,0,1,0
2,A549/ATCC,58,1,0,0,0,0,1,0
3,ACHN,22,1,0,0,0,0,1,0
4,CCRF-CEM,4,1,0,0,0,1,0,0


In [14]:
cellLine_info_full.dtypes

cellName                     string
age                           int64
cat_Cancer cell line          uint8
cat_Embryonic stem cell       uint8
cat_Hybridoma                 uint8
cat_Transformed cell line     uint8
gender_Female                 uint8
gender_Male                   uint8
gender_Sex unspecified        uint8
dtype: object